## Imports

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization

Using TensorFlow backend.


In [3]:
import tensorflow as tf

In [4]:
from os import system as sys

In [5]:
from sklearn.metrics import confusion_matrix

In [6]:
from time import clock

In [7]:
import time
random.seed(int(time.time()))

In [8]:
wavelen=3500

In [9]:
from numpy.fft import rfft, irfft

### Essential Defs

In [10]:
def model_gen():
    model = Sequential()

    model.add(BatchNormalization())

    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(128, 3, activation='relu', padding='same'))
    model.add(Conv1D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [11]:
def shuffle_in_unison(a, b):
    rng_state = random.get_state()
    random.shuffle(a)
    random.set_state(rng_state)
    random.shuffle(b)

In [12]:
def fft_trapezoid(rise_time, flat_top , tau):
    length = wavelen
    length = length+2*rise_time+flat_top;
    p = zeros(length)
    s = zeros(length)
    input2 = zeros(length)
    p[0] = s[0] = input2[0] = 0.;
    for i in range(1,length):
        input2[i] = s[i] = 0.;
    input2[1]=1.;
    tau = 1/(exp(1./tau)-1);
    for i in range(1,length):
        if i>=2*rise_time+flat_top:
            d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]+input2[i-2*rise_time-flat_top]
        else:
            if i>=rise_time+flat_top:
                d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]
            else:
                if i>=rise_time:
                    d = input2[i]-input2[i-rise_time]
                else:
                    d = input2[i];
        p[i] = p[i-1]+d;
        s[i] = s[i-1]+p[i]+tau*d;
    for i in range(length):
        s[i] = s[i]/(rise_time*tau);
    
    res = fft.rfft(s)
    return res[:-(flat_top+rise_time)]

In [13]:
def format_data(X,y):
    trap=fft_trapezoid(50,0,1250)
    for i in range(len(X)):
        X[i] = irfft(trap*rfft(X[i]))
    shuffle_in_unison(X,y)
    X = X.reshape(len(X),wavelen,1)
    return X,y

In [14]:
def gen_data():
    sys('../data_generator/bin/pulse_generation > my_data.dat')
    my_data = loadtxt('my_data.dat')
    my_labels = append(zeros(int(len(my_data)/2)),ones(int(len(my_data)/2)))
    my_data, my_labels = format_data(my_data, my_labels)
    sys('rm my_data.dat')
    return my_data, my_labels

### Comparison

In [15]:
model = model_gen()

In [16]:
X, y = gen_data()

In [17]:
keras.backend.get_session().run(tf.local_variables_initializer())

In [18]:
model.fit(X, y, batch_size=64, epochs=3, verbose=True)    

Epoch 1/3
36000/36000 [==============================] - 54s 1ms/step - loss: 0.5728 - acc: 0.6761
Epoch 2/3
36000/36000 [==============================] - 51s 1ms/step - loss: 0.3662 - acc: 0.8512
Epoch 3/3
36000/36000 [==============================] - 51s 1ms/step - loss: 0.3190 - acc: 0.8741


In [19]:
X, y = gen_data()

In [20]:
model.fit(X, y, batch_size=64, epochs=2, verbose=True)    

Epoch 1/2
36000/36000 [==============================] - 51s 1ms/step - loss: 0.2899 - acc: 0.8899
Epoch 2/2
36000/36000 [==============================] - 52s 1ms/step - loss: 0.2713 - acc: 0.8986


In [21]:
X, y = gen_data()

In [22]:
model.fit(X, y, batch_size=64, epochs=1, verbose=True)    

Epoch 1/1
36000/36000 [==============================] - 51s 1ms/step - loss: 0.2706 - acc: 0.8975


In [23]:
val_X, val_y = gen_data()

In [24]:
model.evaluate(val_X, val_y, batch_size=64)

36000/36000 [==============================] - 16s 451us/step


[0.24246419955624474, 0.89825]